# loading the dataset
the dataset is being loaded from the .csv file which is from https://data.world/datafiniti/hotel-reviews, and we need to retrieve the column with the name reviews.text, and make the ratings as our target with column name review.rating.
NOTE: A100 RAM was used to run the cells below

In [ ]:
!pip install langdetect
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=0a98c098d9bf2f840f2ad5b62efa9c97ff52663c46ce3e8340a90ddedfd0a53b
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install tensorflow

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/0f/a5/89a9bdae9f241cad167edb6c890ef2b4c842c5ae81058a2fbfd702dba9bf/tensorflow-2.17.0-cp311-cp311-macosx_12_0_arm64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/a2/ad/e0d3c824784ff121c03cc031f944bc7e139a8f1870ffd2845cc2dd76f6c4/absl_py-2.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for astunparse>=1.6.0 from https://files.pythonhosted.org/packages/2b/03/13dde6512ad7b4557eb792fbcf0c653af6076b81e5941d36ec61f7ce6028/astunparse-1.6.3-py2.py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers>=24.3.25 from https://files.pythonhosted.org/packages/41/f0/7e988a019bc54b2dbd0ad4182ef2d53488bb02e58694cd79d61369e85900/flatbuffers-24.3.25-py2.py3-none-any.whl.metadata
  Obtaining dependency information for gast!=0.5.0,!=0.5.1,!=0.5.2,>=0.2.1 from https://files.pythonhosted.org/packages/a3/61/8001b38

In [ ]:
#imports
import pandas as pd
import numpy as np
import re
import langdetect
from langdetect import detect, DetectorFactory
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from nltk.classify import NaiveBayesClassifier
from nltk.classify.util import accuracy as nltk_accuracy
from multiprocessing import Pool

#loading the dataset
reviewsDataset = pd.read_csv('7282_1.csv')
reviewsDataset.head()

,address,categories,city,country,latitude,longitude,name,postalCode,province,reviews.date,reviews.dateAdded,reviews.doRecommend,reviews.id,reviews.rating,reviews.text,reviews.title,reviews.userCity,reviews.username,reviews.userProvince
0,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2013-09-22T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,4.0,Pleasant 10 min walk along the sea front to th...,Good location away from the crouds,NaN,Russ (kent),NaN
1,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2015-04-03T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,Really lovely hotel. Stayed on the very top fl...,Great hotel with Jacuzzi bath!,NaN,A Traveler,NaN
2,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2014-05-13T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,Ett mycket bra hotell. Det som drog ner betyge...,Lugnt l��ge,NaN,Maud,NaN
3,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2013-10-27T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,We stayed here for four nights in October. The...,Good location on the Lido.,NaN,Julie,NaN
4,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2015-03-05T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,We stayed here for four nights in October. The...,������ ���������������,NaN,sungchul,NaN


In [ ]:
reviewsDataset['reviews.text'].count()

35888

In [ ]:
#downliading the nltk library data files for tokenization
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


#initializing lemmatizer and setting English stop words set to variable stop_words
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# data cleaning, tokenization
creating a function to check whether the language is indeed english, cheking with RegEx whether the review is symbols and gibberish, and remove any review less than 4 characters.
We also tokenize, and lemmatize the words. which is a NLP technique

In [ ]:
#setting seed 0 for langdetect to ensure reproducibility
langdetect.DetectorFactory.seed = 0


#function to check whether the reviews is english or not
def is_english(review):
    try:
        return detect(review) == 'en'
    except langdetect.lang_detect_exception.LangDetectException:
        return False



#function to clean up the reviews, with regex and applying the is_english function, also checking if the review is less than 4 characters
def clean_review(review):
    # Remove non-English reviews
    if not is_english(review):
        return None
    # Remove symbols and gibberish
    if re.match(r'^[^a-zA-Z0-9]+$', review):
        return None
    # Remove very short reviews
    if len(review) < 4:
        return None

    # Tokenize the review
    words = word_tokenize(review)
    # Remove stop words and lemmatize
    cleaned_words = [lemmatizer.lemmatize(word.lower()) for word in words if word.lower() not in stop_words and word.isalpha()]

    return ' '.join(cleaned_words)




In [ ]:
# Apply cleaning function to reviews.text and keep track of valid indices
valid_indices = []
cleaned_reviews = []

for idx, review in enumerate(reviewsDataset['reviews.text'].fillna('')):
    cleaned_review = clean_review(review)
    if cleaned_review:
        valid_indices.append(idx)
        cleaned_reviews.append(cleaned_review)

In [ ]:
# Filter the dataset to keep only valid rows
reviewsDataset = reviewsDataset.iloc[valid_indices].copy()
reviewsDataset['reviews.text'] = cleaned_reviews

# Combine cleaned reviews.text with reviews.title
reviewsDataset['combined_reviews'] = reviewsDataset['reviews.title'].astype(str) + ". " + reviewsDataset['reviews.text']

# Extract combined reviews and ratings
reviews = np.array(reviewsDataset['combined_reviews'])
ratings = np.array(reviewsDataset['reviews.rating'])

# Convert ratings to binary labels (positive: rating >= 3, negative: rating < 3)
ratings = np.array([1 if rating >= 3 else 0 for rating in ratings])

# Verify the length of reviews and ratings
print(f"Number of reviews: {len(reviews)}")
print(f"Number of ratings: {len(ratings)}")

Number of reviews: 33656
Number of ratings: 33656


Result of cleaning: We have eliminated 2232 rows


now we split the data to train, test.

In [ ]:
# Split the data into training and testing sets

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(reviews, ratings, test_size=0.25, random_state=42)

In [ ]:
#checking out the shape of the train and test set
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(25242,)
(25242,)
(8414,)
(8414,)


In [ ]:
# Function to extract features from reviews
def extract_features(review):
    words = word_tokenize(review)
    return dict([(word, True) for word in words if word.isalpha()])

In [ ]:
# Prepare training data for the NLTK model
training_data = [(extract_features(review), label) for review, label in zip(X_train, y_train)]

In [ ]:
# Train the NaiveBayesClassifier
classifier = nltk.NaiveBayesClassifier.train(training_data)


In [ ]:
# Prepare test data
test_data = [(extract_features(review), label) for review, label in zip(X_test, y_test)]

In [ ]:
# Evaluate the classifier
y_pred = [classifier.classify(features) for features, _ in test_data]
accuracy = accuracy_score(y_test, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='binary')

In [ ]:
print(f"Test Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

Test Accuracy: 0.79
Precision: 0.96
Recall: 0.77
F1-Score: 0.85


In [ ]:
# Test the prediction function
sample_review = reviewsDataset['combined_reviews'].iloc[0]
cleaned_sample_review = clean_review(sample_review)
sample_features = extract_features(cleaned_sample_review)
predicted_label = classifier.classify(sample_features)

print(f"Review: {sample_review}")
print(f"Predicted Sentiment: {'positive' if predicted_label == 1 else 'negative'}")
print(f"Actual Rating Sentiment: {'positive' if ratings[0] == 1 else 'negative'}")

Review: Good location away from the crouds. pleasant min walk along sea front water bus restaurant etc hotel comfortable breakfast good quite variety room aircon work well take mosquito repelant
Predicted Sentiment: positive
Actual Rating Sentiment: positive


# using LSTM

In [ ]:
#loading the dataset
reviewsDataset = pd.read_csv('7282_1.csv')
reviewsDataset.head()

,address,categories,city,country,latitude,longitude,name,postalCode,province,reviews.date,reviews.dateAdded,reviews.doRecommend,reviews.id,reviews.rating,reviews.text,reviews.title,reviews.userCity,reviews.username,reviews.userProvince
0,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2013-09-22T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,4.0,Pleasant 10 min walk along the sea front to th...,Good location away from the crouds,NaN,Russ (kent),NaN
1,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2015-04-03T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,Really lovely hotel. Stayed on the very top fl...,Great hotel with Jacuzzi bath!,NaN,A Traveler,NaN
2,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2014-05-13T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,Ett mycket bra hotell. Det som drog ner betyge...,Lugnt l��ge,NaN,Maud,NaN
3,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2013-10-27T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,We stayed here for four nights in October. The...,Good location on the Lido.,NaN,Julie,NaN
4,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2015-03-05T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,We stayed here for four nights in October. The...,������ ���������������,NaN,sungchul,NaN


In [ ]:
# Apply cleaning function to reviews.text and keep track of valid indices
valid_indices = []
cleaned_reviews = []

for idx, review in enumerate(reviewsDataset['reviews.text'].fillna('')):
    cleaned_review = clean_review(review)
    if cleaned_review:
        valid_indices.append(idx)
        cleaned_reviews.append(cleaned_review)

In [ ]:
# Filter the dataset to keep only valid rows
reviewsDataset = reviewsDataset.iloc[valid_indices].copy()
reviewsDataset['reviews.text'] = cleaned_reviews

# Combine cleaned reviews.text with reviews.title
reviewsDataset['combined_reviews'] = reviewsDataset['reviews.title'].astype(str) + ". " + reviewsDataset['reviews.text']

# Extract combined reviews and ratings
reviews = np.array(reviewsDataset['combined_reviews'])
ratings = np.array(reviewsDataset['reviews.rating'])

# Convert ratings to binary labels (positive: rating >= 3, negative: rating < 3)
ratings = np.array([1 if rating >= 3 else 0 for rating in ratings])

# Verify the length of reviews and ratings
print(f"Number of reviews: {len(reviews)}")
print(f"Number of ratings: {len(ratings)}")

Number of reviews: 33656
Number of ratings: 33656


In [ ]:
# Split the data into training and testing sets

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(reviews, ratings, test_size=0.25, random_state=42)

In [ ]:
#checking out the shape of the train and test set
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(25242,)
(25242,)
(8414,)
(8414,)


Now we prepare features, prepare training data, train Naive Bayes Classifier (it will take 3-4 minutes probably)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense,Flatten, SpatialDropout1D


# Tokenize and pad the text sequences
tokenizer = Tokenizer(num_words=5000, lower=True, split=' ')
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

max_seq_len = 500
X_train = pad_sequences(X_train, maxlen=max_seq_len)
X_test = pad_sequences(X_test, maxlen=max_seq_len)






In [ ]:
# Build the LSTM model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=max_seq_len))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
#now we compile the model and fit the data into the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
# Train the model (might take more than 10 minutes)
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test), verbose=2)


Epoch 1/5
395/395 - 542s - loss: 0.3370 - accuracy: 0.8575 - val_loss: 0.2808 - val_accuracy: 0.8833 - 542s/epoch - 1s/step
Epoch 2/5
395/395 - 528s - loss: 0.2483 - accuracy: 0.8999 - val_loss: 0.2621 - val_accuracy: 0.8908 - 528s/epoch - 1s/step
Epoch 3/5
395/395 - 526s - loss: 0.2159 - accuracy: 0.9119 - val_loss: 0.2674 - val_accuracy: 0.8838 - 526s/epoch - 1s/step
Epoch 4/5
395/395 - 527s - loss: 0.1892 - accuracy: 0.9243 - val_loss: 0.2791 - val_accuracy: 0.8902 - 527s/epoch - 1s/step
Epoch 5/5
395/395 - 526s - loss: 0.1670 - accuracy: 0.9327 - val_loss: 0.3199 - val_accuracy: 0.8820 - 526s/epoch - 1s/step


now we can evaluate the model to get the accuracy, f1-score, precision and recall.

In [ ]:
#evaluate the model
from sklearn.metrics import precision_recall_fscore_support, accuracy_score


# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Test Accuracy: {accuracy:.2f}")


# Predict the sentiment on the test set
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob >= 0.5).astype(int)


# Calculate precision, recall, and f1-score
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='binary')

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

263/263 - 35s - loss: 0.3199 - accuracy: 0.8820 - 35s/epoch - 134ms/step
Test Accuracy: 0.88
263/263 [==============================] - 36s 136ms/step
Precision: 0.93
Recall: 0.92
F1-Score: 0.93


now we run a sample test to see how it will predict an unseen review

In [ ]:
# Function to predict sentiment
def predict_sentiment(review):
    seq = tokenizer.texts_to_sequences([review])
    padded = pad_sequences(seq, maxlen=max_seq_len)
    pred = model.predict(padded)
    return 'positive' if pred >= 0.5 else 'negative'

# Test the prediction function
sample_review = reviewsDataset['combined_reviews'].iloc[0]
print(f"Review: {sample_review}")
print(f"Predicted Sentiment: {predict_sentiment(sample_review)}")
print(f"Actual Rating Sentiment: {'positive' if ratings[0] == 1 else 'negative'}")

Review: Good location away from the crouds. pleasant min walk along sea front water bus restaurant etc hotel comfortable breakfast good quite variety room aircon work well take mosquito repelant
1/1 [==============================] - 0s 171ms/step
Predicted Sentiment: positive
Actual Rating Sentiment: positive


What if instead of LSTM based Neural network we used a simple ANN? let's explore the second model


# using an ANN

In [ ]:
#loading the dataset
reviewsDataset = pd.read_csv('datafiniti-hotel-reviews/7282_1.csv')
reviewsDataset.head()

,address,categories,city,country,latitude,longitude,name,postalCode,province,reviews.date,reviews.dateAdded,reviews.doRecommend,reviews.id,reviews.rating,reviews.text,reviews.title,reviews.userCity,reviews.username,reviews.userProvince
0,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2013-09-22T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,4.0,Pleasant 10 min walk along the sea front to th...,Good location away from the crouds,NaN,Russ (kent),NaN
1,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2015-04-03T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,Really lovely hotel. Stayed on the very top fl...,Great hotel with Jacuzzi bath!,NaN,A Traveler,NaN
2,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2014-05-13T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,Ett mycket bra hotell. Det som drog ner betyge...,Lugnt l��ge,NaN,Maud,NaN
3,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2013-10-27T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,We stayed here for four nights in October. The...,Good location on the Lido.,NaN,Julie,NaN
4,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2015-03-05T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,We stayed here for four nights in October. The...,������ ���������������,NaN,sungchul,NaN


In [ ]:
# Apply cleaning function to reviews.text and keep track of valid indices
valid_indices = []
cleaned_reviews = []

for idx, review in enumerate(reviewsDataset['reviews.text'].fillna('')):
    cleaned_review = clean_review(review)
    if cleaned_review:
        valid_indices.append(idx)
        cleaned_reviews.append(cleaned_review)

In [ ]:
# Filter the dataset to keep only valid rows
reviewsDataset = reviewsDataset.iloc[valid_indices].copy()
reviewsDataset['reviews.text'] = cleaned_reviews

# Combine cleaned reviews.text with reviews.title
reviewsDataset['combined_reviews'] = reviewsDataset['reviews.title'].astype(str) + ". " + reviewsDataset['reviews.text']

# Extract combined reviews and ratings
reviews = np.array(reviewsDataset['combined_reviews'])
ratings = np.array(reviewsDataset['reviews.rating'])

# Convert ratings to binary labels (positive: rating >= 3, negative: rating < 3)
ratings = np.array([1 if rating >= 3 else 0 for rating in ratings])

# Verify the length of reviews and ratings
print(f"Number of reviews: {len(reviews)}")
print(f"Number of ratings: {len(ratings)}")

Number of reviews: 33656
Number of ratings: 33656


In [ ]:
# Split the data into training and testing sets

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(reviews, ratings, test_size=0.25, random_state=42)

In [ ]:
#checking out the shape of the train and test set
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(25242,)
(25242,)
(8414,)
(8414,)


In [ ]:
# Tokenize and pad the text sequences
tokenizer = Tokenizer(num_words=5000, lower=True, split=' ')
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

max_seq_len = 500
X_train = pad_sequences(X_train, maxlen=max_seq_len)
X_test = pad_sequences(X_test, maxlen=max_seq_len)

In [ ]:
# Build the ANN model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=max_seq_len))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


/Users/bardiadehbasti/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
#compiling the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test), verbose=2)

Epoch 1/5
395/395 - 26s - 65ms/step - accuracy: 0.8325 - loss: 0.4658 - val_accuracy: 0.8804 - val_loss: 0.2781
Epoch 2/5
395/395 - 23s - 57ms/step - accuracy: 0.9035 - loss: 0.2332 - val_accuracy: 0.8795 - val_loss: 0.2801
Epoch 3/5
395/395 - 23s - 57ms/step - accuracy: 0.9519 - loss: 0.1328 - val_accuracy: 0.8708 - val_loss: 0.3453
Epoch 4/5
395/395 - 23s - 58ms/step - accuracy: 0.9807 - loss: 0.0631 - val_accuracy: 0.8690 - val_loss: 0.3780
Epoch 5/5
395/395 - 23s - 57ms/step - accuracy: 0.9883 - loss: 0.0409 - val_accuracy: 0.8633 - val_loss: 0.4275


In [ ]:
#evaluating the model with accuracy and f1-score, precision, recall

loss, accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Test Accuracy: {accuracy:.2f}")

# Predict the sentiment on the test set
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob >= 0.5).astype(int)

# Calculate precision, recall, and f1-score
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='binary')

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

263/263 - 2s - 6ms/step - accuracy: 0.8633 - loss: 0.4275
Test Accuracy: 0.86
263/263 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step
Precision: 0.90
Recall: 0.93
F1-Score: 0.92


In [ ]:
#testing an unseen review to see how it predicts it
# Function to predict sentiment
def predict_sentiment(review):
    seq = tokenizer.texts_to_sequences([review])
    padded = pad_sequences(seq, maxlen=max_seq_len)
    pred = model.predict(padded)
    return 'positive' if pred >= 0.5 else 'negative'

# Test the prediction function
sample_review = reviewsDataset['combined_reviews'].iloc[0]
print(f"Review: {sample_review}")
print(f"Predicted Sentiment: {predict_sentiment(sample_review)}")
print(f"Actual Rating Sentiment: {'positive' if ratings[0] == 1 else 'negative'}")

Review: Good location away from the crouds. pleasant min walk along sea front water bus restaurant etc hotel comfortable breakfast good quite variety room aircon work well take mosquito repelant
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Predicted Sentiment: positive
Actual Rating Sentiment: positive


# using RoBERTa

In [ ]:
!pip install transformers

In [ ]:
!pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

In [ ]:
#redoing all the previous steps to load dataset, clean the data, with a different tokenization method, (using AutoTokenize which is from the transformers library)

#loading the dataset
reviewsDataset = pd.read_csv('7282_1.csv')
reviewsDataset.head()

,address,categories,city,country,latitude,longitude,name,postalCode,province,reviews.date,reviews.dateAdded,reviews.doRecommend,reviews.id,reviews.rating,reviews.text,reviews.title,reviews.userCity,reviews.username,reviews.userProvince
0,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2013-09-22T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,4.0,Pleasant 10 min walk along the sea front to th...,Good location away from the crouds,NaN,Russ (kent),NaN
1,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2015-04-03T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,Really lovely hotel. Stayed on the very top fl...,Great hotel with Jacuzzi bath!,NaN,A Traveler,NaN
2,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2014-05-13T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,Ett mycket bra hotell. Det som drog ner betyge...,Lugnt l��ge,NaN,Maud,NaN
3,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2013-10-27T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,We stayed here for four nights in October. The...,Good location on the Lido.,NaN,Julie,NaN
4,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2015-03-05T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,We stayed here for four nights in October. The...,������ ���������������,NaN,sungchul,NaN


In [ ]:
# Apply cleaning function to reviews.text and keep track of valid indices
valid_indices = []
cleaned_reviews = []

for idx, review in enumerate(reviewsDataset['reviews.text'].fillna('')):
    cleaned_review = clean_review(review)
    if cleaned_review:
        valid_indices.append(idx)
        cleaned_reviews.append(cleaned_review)

In [ ]:
# Filter the dataset to keep only valid rows
reviewsDataset = reviewsDataset.iloc[valid_indices].copy()
reviewsDataset['reviews.text'] = cleaned_reviews

# Combine cleaned reviews.text with reviews.title
reviewsDataset['combined_reviews'] = reviewsDataset['reviews.title'].astype(str) + ". " + reviewsDataset['reviews.text']

# Extract combined reviews and ratings
reviews = np.array(reviewsDataset['combined_reviews'])
ratings = np.array(reviewsDataset['reviews.rating'])

# Convert ratings to binary labels (positive: rating >= 3, negative: rating < 3)
ratings = np.array([1 if rating >= 3 else 0 for rating in ratings])

# Verify the length of reviews and ratings
print(f"Number of reviews: {len(reviews)}")
print(f"Number of ratings: {len(ratings)}")

Number of reviews: 33656
Number of ratings: 33656


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(reviews, ratings, test_size=0.25, random_state=42)

In [ ]:
#checking out the shape of the train and test set
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(25242,)
(25242,)
(8414,)
(8414,)


now we load the RoBERTa model from hugging face, and the tokenizer from AutoTokenizer.
here is the link to the model used:
https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest

In [ ]:
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#we create a function to encode the reviews, using the tokenizer we initiated above

def encode_reviews(reviews):
    return tokenizer(reviews.tolist(), truncation=True, padding=True, max_length=128, return_tensors='pt')


X_train_enc = encode_reviews(X_train)
X_test_enc = encode_reviews(X_test)

In [ ]:
#create a function that makes predition using the model made above

def predict_sentiment_roberta(encoded_reviews):
    with torch.no_grad():
        outputs = model(**encoded_reviews)
        logits = outputs.logits
        predictions = torch.nn.functional.softmax(logits, dim=-1)
    return predictions

In [ ]:
#predicting sentiments for the test set

import torch

y_pred_probs = predict_sentiment_roberta(X_test_enc)
y_pred = torch.argmax(y_pred_probs, axis=1).numpy()

In [ ]:
#since the model maps the models to negative, neutral and positive, we need to map them back to a binary classfication, setting neutral to negative

# Map the three classes to binary labels (0: negative or neutral, 1: positive)
y_pred_binary = np.where(y_pred == 2, 1, 0)  # Class 2 (positive) to 1, else to 0

In [ ]:
print("Unique labels in y_pred:", np.unique(y_pred))
print("Unique labels in y_pred_binary:", np.unique(y_pred_binary))

Unique labels in y_pred: [0 1 2]
Unique labels in y_pred_binary: [0 1]


In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
# Calculate accuracy, precision, recall, and F1-score
accuracy = accuracy_score(y_test, y_pred_binary)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred_binary, average='binary')


In [ ]:
print(f"Test Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

Test Accuracy: 0.83
Precision: 0.94
Recall: 0.83
F1-Score: 0.88


In [ ]:
#testing the prediction function

In [ ]:
# redefining the function in order to remove .tolist().
def encode_reviews2(reviews):
    return tokenizer(reviews, truncation=True, padding=True, max_length=128, return_tensors='pt')

sample_review = reviewsDataset['combined_reviews'].iloc[0]
encoded_sample_review = encode_reviews2([sample_review])
predicted_sentiment = predict_sentiment_roberta(encoded_sample_review)
predicted_label = torch.argmax(predicted_sentiment, axis=1).numpy()[0]
predicted_label_binary = 1 if predicted_label == 2 else 0  # Class 2 (positive) to 1, else to 0
print(f"Review: {sample_review}")
print(f"Predicted Sentiment: {'positive' if predicted_label_binary == 1 else 'negative'}")
print(f"Actual Rating Sentiment: {'positive' if ratings[0] == 1 else 'negative'}")

Review: Good location away from the crouds. pleasant min walk along sea front water bus restaurant etc hotel comfortable breakfast good quite variety room aircon work well take mosquito repelant
Predicted Sentiment: positive
Actual Rating Sentiment: positive
